In [1]:
import numpy as np
from numpy.random import permutation

import os
import glob
import cv2
import math
import sys

import pandas as pd

from keras.models import Sequential
from keras.models import model_from_json
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD
from keras.utils import np_utils

import tensorflow as tf

# VGG 16
from keras.applications import VGG16

# ResNet152
from keras.applications import ResNet152

import warnings
warnings.filterwarnings('ignore')

In [2]:
# GPU 강제 할당
# 사용 가능한 GPU 목록을 가져온다.
# Physical devices cannot be modified after being initialized : 이 문구가 나오면 재시작
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        # 필요한 만큼만 메모리를 사용할 수 있도록 설정한다.
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
# 랜덤시드 
np.random.seed(1)

In [4]:
# 사용하는 이미지 사이즈
img_rows = 224
img_cols = 224

### 학습

In [5]:
def resnet152_model():
    model1 = ResNet152(weights='imagenet', include_top = False,
                      input_shape=(img_rows, img_cols, 3))
    model1.trainable = False
    # model1.summary()
    
    # 모델 생성
    model = Sequential()
    
    # ResNet152 모델을 추가한다.
    model.add(model1)
    
    model.add(Flatten())
    
    model.add(Dense(6, activation='softmax'))
    
    # 컴파일 
    model.compile(optimizer='adam', loss='categorical_crossentropy',
                 metrics=['accuracy'])
    
    model.summary()
    
    return model

In [6]:
resnet152_model()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet152 (Functional)       (None, 7, 7, 2048)        58370944  
_________________________________________________________________
flatten (Flatten)            (None, 100352)            0         
_________________________________________________________________
dense (Dense)                (None, 6)                 602118    
Total params: 58,973,062
Trainable params: 602,118
Non-trainable params: 58,370,944
_________________________________________________________________


In [7]:
# 이미지 1장을 읽어오고 리사이징한다.
def get_img(path):
    img = cv2.imread(path)
    resized = cv2.resize(img, (img_rows, img_cols))
    return resized

In [8]:
# 학습 데이터를 읽어오는 함수
def read_train_data(ho=0, kind='train'):
    # 학습용 입력데이터를 담을 리스트
    train_data = []
    # 학습용 결과데이터를 담을 리스트
    train_target = []
    
    # 결과 종류의 수만큼 반복한다.(비행기, 오토바이, 등등)
    for j in range(0,6):
        # 이미지의 경로
        path = 'data/Caltech-101/'
        path += f'{kind}/{ho}/*/{j}/*.jpg' # ex) train/0/0/*.jpg
        
        # 파일 목록을 가져온다.
        files = sorted(glob.glob(path))
        # print(files)
        
        # 파일의 수만큼 반복한다.
        for f1 in files:
            # 파일 이름을 가져온다.
            file_base = os.path.basename(f1)
            
            # 이미지 1장을 읽어온다.
            img = get_img(f1)
            img = np.array(img, dtype=np.float32)
            
            # 데이터 정규화
            img -= np.mean(img)
            img /= np.std(img)
            
            # 리스트에 담는다.
            train_data.append(img)
            train_target.append(j)
    
    # 읽어들인 데이터를 numpy의 array로 변환
    train_data = np.array(train_data, dtype=np.float32)
    train_target = np.array(train_target, dtype=np.uint8)
    
    # target을 원핫 인코딩한다.
    # 예) 1 -> 0,1,0,0,0,0
    train_target = np_utils.to_categorical(train_target, 6)
    
    # 데이터를 섞는다.
    perm = permutation(len(train_target))
    train_data = train_data[perm]
    train_target = train_target[perm]
    
    return train_data, train_target

In [9]:
# 모델의 구조와 가중치를 저장한다.
def save_model(model, ho, modelStr=''):
    
    # 모델 객체를 json 형식으로 변환한다.
    json_string = model.to_json()
    
    # cache 폴더가 없으면 만들어준다.
    if not os.path.exists('cache'):
        os.makedirs('cache')
        
    # 모델 구조를 저장하기 위한 파일명
    json_name = f'architecture_{modelStr}_{ho}.json'
    
    # 모델 구조를 저장한다.
    with open(os.path.join('cache', json_name),'w') as fp:
        fp.write(json_string)

In [10]:
# 학습 함수
def run_train(modelStr=''):
    # HoldOut을 두번 수행한다.
    for ho in range(2):
        # 모델을 생성한다.
        model = resnet152_model()
        
        # 학습 데이터를 읽어온다. 함수호출
        t_data, t_target = read_train_data(ho,'train') 
        # 검증 데이터를 읽어온다.
        v_data, v_target = read_train_data(ho,'valid')
        
        # 매 epoch마다 모델을 저장하기 위한 callback을 생성한다.
        # save_best_only = True : 손실률을 비교하여 가장 적은 모델을 저장해나감
        cp = ModelCheckpoint(f'cache/model_weights_{modelStr}_{ho}_' + '{epoch:02d}.h5',
                            monitor='val_loss', save_best_only=True)
        
        # train 실행
        model.fit(t_data, t_target, batch_size=16, epochs=40,
                 validation_data=(v_data, v_target), shuffle=True, callbacks=[cp])
        
        # 모델 구조 저장
        save_model(model, ho, modelStr)

---

In [11]:
# test용 이미지를 불러오는 함수
def load_test(test_class, aug_i):
    # 경로
    path = f'data/Caltech-101/test/{aug_i}/{test_class}/*.jpg'
    
    # 파일 목록을 가져온다.
    files = sorted(glob.glob(path))
    # display(files)
    
    # 이미지 데이터를 담을 리스트
    X_test = []
    # 결과데이터를 담을 리스트
    X_test_id = []
    
    # 파일의 수만큼 반복한다.
    for f1 in files:
        # 파일의 이름을 가져온다.
        f1base = os.path.basename(f1)
        
        # 이미지 데이터를 가져온다.
        img = get_img(f1)
        img = np.array(img, dtype=np.float32)
        
        # 정규화
        img -= np.mean(img)
        img /= np.std(img)
        
        # 담는다.
        X_test.append(img)
        X_test_id.append(f1base)
     
    # 읽어들인 데이터를 ndarray 최종변환
    test_data = np.array(X_test, dtype=np.float32)
    
    return test_data, X_test_id

In [12]:
# 저장된 모델을 복원하는 함수
def read_model(ho, modelStr='', epoch='00'):
    # 모델 구조의 파일명
    json_name = f'cache/architecture_{modelStr}_{ho}.json'
    
    # 모델 가중치 파일명
    weight_name = f'cache/model_weights_{modelStr}_{ho}_{epoch}.h5'
    
    # 모델 구조를 json으로부터 읽어 복원한다.
    model = model_from_json(open(json_name).read())
    
    # 복원된 모델에 가중치값을 셋팅한다.
    model.load_weights(weight_name)
    
    return model

In [13]:
# 예측 함수
def run_test(modelStr, epoch1, epoch2):
    # 결과 데이터 불러오기
    columns = []
    
    # 파일에서 데이터를 읽어온다.
    with open ('data/Caltech-101/label.csv','r') as fp:
        line = fp.readline()
    display(line)
    
    # 쉼표 (,)를 기준으로 잘라낸다.
    sp = line.split(',')
    # 잘라낸 문장만큼 반복한다.
    for c1 in sp:
        # 콜론(:)을 기준으로 잘라낸다.
        sp2 = c1.split(':')
        columns.append(sp2[1])
        
    print(columns)
            
    # 테스트 데이터가 각 클래스로 나누어지므로
    # 1 클래스씩 읽어서 예측을 실행한다.
    for test_class in range(0, 6):
        # 예측된 결과를 담을 리스트
        yfull_test = []
        
        # 하나의 이미지가 5번 변환되어 있으므로 이 수만큼 반복한다.
        for aug_i in range(0,5):
            # 예측할 이미지 데이터를 불러온다.
            test_data, test_id = load_test(test_class, aug_i)
            # display(test_id)
            
            # 홀드아웃 수만큼 반복한다.
            for ho in range(2):
                # 모델을 복원한다.
                if ho == 0:
                    n_epoch = epoch1
                else:
                    n_epoch = epoch2
                    
                model = read_model(ho, modelStr, n_epoch)
                # display(model)
                
                # 예측을 실행한다.
                test_p = model.predict(test_data, batch_size=32, verbose=1)
                
                # 예측된 결과를 담는다.
                yfull_test.append(test_p)
                
        # 예측(상위 10개) 결과의 평균을 구한다.
        test_res = np.array(yfull_test[0])
        for i in range(1,10):
            test_res = np.array(yfull_test[1])
            
        test_res /= 10
        
        # 예측결과와 클래스명, 이미지명을 합함
        result1 = pd.DataFrame(test_res, columns=columns)
        result1.loc[:,'img'] = pd.Series(test_id, index = result1.index)
        
        # 저장한다.
        if not os.path.exists('subm'):
            os.makedirs('subm')
        sub_file = f'subm/result_{modelStr}_{test_class}.csv'
        result1.to_csv(sub_file, index=False)
        
        # 위에서 구한 예측 정확도가 가장 높은것을 가져온다.
        a1 = np.argmax(test_res, axis=1)
        
        # 정답수와 오답수를 구한다.
        one_column = np.where(a1 == test_class)
        
        print(f'정답수 : {len(one_column[0])}')
        print(f'오답수 : {test_res.shape[0] - len(one_column[0])}')

In [14]:
# 실행모드
# run_type = 'train'
run_type = 'test'

In [15]:
# GPU 실행
# GPU 메모리 부족 에러 Failed to get convolution algorithm
# run_test('VGG16','01','01') : '01'은 생성된 모델 번호에 맞춰야함
# with tf.device('/CPU:0'):
with tf.device('/GPU:0'):
    if run_type == 'train':
        run_train('ResNet152')
    elif run_type == 'test':
        run_test('ResNet152','08','01')

'0:airplanes,1:Motorbikes,2:Faces_easy,3:watch,4:Leopards,5:bonsai'

['airplanes', 'Motorbikes', 'Faces_easy', 'watch', 'Leopards', 'bonsai']
20/20 [==============================] - 7s 245ms/step
정답수 : 521
오답수 : 119
20/20 [==============================] - 7s 225ms/step
정답수 : 350
오답수 : 288
11/11 [==============================] - 5s 233ms/step
정답수 : 162
오답수 : 185
6/6 [==============================] - 4s 263ms/step
정답수 : 121
오답수 : 70
5/5 [==============================] - 4s 270ms/step
정답수 : 111
오답수 : 49
4/4 [==============================] - 3s 217ms/step
정답수 : 52
오답수 : 50


모델 성능이 만족하지 않다면 VGG 전결합층을 추가하는 방법도 있다.
원본이미지는 잘 판별하나 변형된 이미지는 판별하지 못함